In [35]:
import pandas as pd
import numpy as np
import string
import nltk
import numpy as np

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

df = pd.read_csv("../data/fulldata.csv")
df2 = pd.read_csv("../data/6_14_pull.csv")

df = pd.concat([df,df2])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from Cleaning_Topic import lowercase, unpunctuate, tokenize, lemmatize, cleaning_and_prep

In [3]:
X, y  = cleaning_and_prep(df)

C:\Users\Andrew\Documents\LHL\PoliticalNLP\python\Cleaning_Topic.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Political Lean'] = df['Political Lean'].replace('Liberal',1)
C:\Users\Andrew\Documents\LHL\PoliticalNLP\python\Cleaning_Topic.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Political Lean'] = df['Political Lean'].replace('Conservative',0)


In [4]:
from gensim import corpora

# Convert tokenized sentences to a list of words
tokenized_sentences = X['Title'].tolist()
dictionary = corpora.Dictionary(tokenized_sentences)

In [5]:
corpus = [dictionary.doc2bow(doc) for doc in tokenized_sentences]

In [6]:
from gensim.models import LdaModel
from gensim.models import CoherenceModel

In [7]:
num_topics = 12
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

In [8]:
topics = lda_model.show_topics(num_topics=num_topics, formatted=False)

In [9]:
for topic in topics:
    print(f"Topic {topic[0]}:")
    words = [word[0] for word in topic[1]]
    print(words)

Topic 0:
['school', 'public', 'book', 'ban', 'child', 'state', 'movement', 'right', 'event', 'politics']
Topic 1:
['government', 'system', 'elon', 'tucker', 'business', 'worker', 'musk', 'carlson', 'palestinian', 'pandemic']
Topic 2:
['police', 'pride', 'republican', 'fbi', 'ted', 'law', 'ballot', 'cruz', 'lawmaker', 'stop']
Topic 3:
['opinion', 'republican', 'vote', 'senate', 'plan', 'bad', '—', 'gop', 'kill', 'member']
Topic 4:
['ukraine', 'russia', 'war', 'putin', 'russian', 'act', 'invasion', 'force', 'report', 'fox']
Topic 5:
['2022', 'work', 'sen', 'covid', 'cpac', 'state', 'strike', 'midterm', 'democrat', 'many']
Topic 6:
['people', 'capitalism', 'socialism', 'left', 'american', 'wage', 'time', 'wrong', 'want', 'death']
Topic 7:
['house', 'white', '6', 'democrat', 'poll', 'former', 'gop', 'president', 'republican', 'donald']
Topic 8:
['social', 'democracy', 'communist', 'socialist', 'state', 'union', 'economy', 'america', 'american', 'free']
Topic 9:
['desantis', 'court', 'elect

In [24]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_sentences, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()

def topic_assignment(text):
    bow_vector = dictionary.doc2bow(text)
    topic_distribution = lda_model.get_document_topics(bow_vector)
    topic_assignment = max(topic_distribution, key=lambda x: x[1])[0]
    return topic_assignment

X['Topic'] = X['Title'].apply(topic_assignment)

In [11]:
coherence_score

0.376168290542803

In [36]:
df = df.merge(X['Topic'], how = 'left', left_index = True, right_index=True)

In [43]:
pd.set_option('display.max_colwidth', None)
df[df['Topic']==1][['Title','Topic']].head(20)

,Title,Topic
8,Every city have to government their resources and give best quality producen to a country environment. And stop financing Political partis which cost 100x govermant spending.,1
8,The IRS Allows Eco-Terrorists to Crowdsource Harassment of Senators | Frontpage Mag,1
24,Alan Kelly resigns as Irish Labour Party leader,1
24,Fighting Woke,1
35,Coverage of China’s Covid-Zero Strategy and Manufacturing Consent,1
35,ISO Opinions,1
65,"Rick Scott (FL) the richest senator, released his 11 point plan. First words are “Our kids will say the pledge of allegiance, salute the Flag”",1
65,Sen. Ron Johnson to Newsmax: Evidence Against Biden Keeps Mounting,1
78,"Russia does not threaten Hungary, lets stay out of the war! Statement of the Hungarian Worker's Party",1
78,Opinion | Trump’s Boxes and Clinton’s Sock Drawer,1
